In [1]:
from IPython.display import display, Markdown
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))
    
%run -i 'implicit.py'

# Crandall-Rabinowitz

We now have a readily callable implicit function theorem, lets put it to work and automate the method of Crandall-Rabinowitz

Crandall-Rabinowitz pertains to the following case:

$$F : \mathbb{R} \times X \rightarrow Y $$

$$ X = \mathbb{R}^n $$

Where we have that $\partial_{x}F$ has Fredholm index 0 and a one-dimensional kernel.

From this we can recover that:

$$dim X = dim Y$$

and:

$$ \text{dimRange} \;\partial_{x}F = n-1 $$

### Changing basis

To begin to untangle the parts we can deal with the implict function theorem, consider the following bases for $X$ and $Y$:

$(u_{1}, \dots, u_{n-1}, \tilde{u}_{n})$ and $(v_{1}, \dots, v_{n-1}, \tilde{v}_{n})$

such that:

$$ \text{Ker} \;\partial_{x}F = \left< \tilde{u}_{n}\right>$$
$$ \text{Range} \;\partial_{x}F = \left< v_{1}, \cdots, v_{n-1} \right>$$

Let $A$ and $B \in \mathcal{L}(\mathbb{R}^n, \mathbb{R}^n)$ be the respective change of basis matrices

See that $$\tilde{F} = B^{-1} \circ F \circ A$$

Now has one input variable acting in the kernel and one output not in the range, (abusing notation here, $A$,$B$ leave the $\mathbb{R}$ input alone)

### Worked example

To build up the method lets use a previous example:

In [2]:
var('x y z')
var('l', latex_name=r'\lambda')
f1(x,y,z,l) = x*sin(sqrt(x^2 + y^2 +  (pi^2)/9)) - y*cos(sqrt(x^2 + y^2 +  (pi^2)/9)) + exp(z) - 1 # zeros when z = 0
f2(x,y,z,l) = x*cos(l) + y*sin(l) + z 
f3(x,y,z,l) = z^2 + 4*z + z*l + x^2 + y^3 # zeros out at when X = 0   [X = <x,y,z>]
lprint(f1)
lprint(f2)
lprint(f3)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -y \cos\left(\frac{1}{3} \, \sqrt{\pi^{2} + 9 \, x^{2} + 9 \, y^{2}}\right) + x \sin\left(\frac{1}{3} \, \sqrt{\pi^{2} + 9 \, x^{2} + 9 \, y^{2}}\right) + e^{z} - 1 $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ x \cos\left({\lambda}\right) + y \sin\left({\lambda}\right) + z $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ y^{3} + x^{2} + {\lambda} z + z^{2} + 4 \, z $

We aim to describe the solution set when $\lambda = -\frac{\pi}{6} $, and determine that it is indeed a bifucation point

But before all that to make things a lot easier lets shift our functions so that we are considering $\lambda = 0$, otherwise the taylor expansions can get very confusing, whatever conclusions we draw about the shifted functions will hold for the originals

In [3]:
old_funcs = [f1,f2,f3]
f1(x,y,z,l) = f1(x,y,z,l-pi/6)
f2(x,y,z,l) = f2(x,y,z,l-pi/6)
f3(x,y,z,l) = f3(x,y,z,l-pi/6)
lprint(f1)
lprint(f2)
lprint(f3)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -y \cos\left(\frac{1}{3} \, \sqrt{\pi^{2} + 9 \, x^{2} + 9 \, y^{2}}\right) + x \sin\left(\frac{1}{3} \, \sqrt{\pi^{2} + 9 \, x^{2} + 9 \, y^{2}}\right) + e^{z} - 1 $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ x \cos\left(-\frac{1}{6} \, \pi + {\lambda}\right) + y \sin\left(-\frac{1}{6} \, \pi + {\lambda}\right) + z $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ y^{3} + x^{2} - \frac{1}{6} \, {\left(\pi - 6 \, {\lambda}\right)} z + z^{2} + 4 \, z $

Now lets proceed, with the potential bifucation point taken to be $\lambda = 0$

In [4]:
J = jacobian([f1,f2,f3],(x,y,z))(x=0,y=0,z=0,l=0)
lprint(J)

$ \left(\begin{array}{rrr}
\frac{1}{2} \, \sqrt{3} & -\frac{1}{2} & 1 \\
\frac{1}{2} \, \sqrt{3} & -\frac{1}{2} & 1 \\
0 & 0 & -\frac{1}{6} \, \pi + 4
\end{array}\right) $

In [5]:
K = J.right_kernel().basis()
lprint(K)

$ \left[\left(1,\,\sqrt{3},\,0\right)\right] $

In [6]:
I = J.image().basis()
lprint(I)

$ \left[\left(1,\,-\frac{1}{3} \, \sqrt{3},\,0\right), \left(0,\,0,\,1\right)\right] $

#### Extending to bases

We can use this trick as per math stackexhange https://math.stackexchange.com/questions/465870/how-to-extend-a-basis

In [7]:
Kperp = matrix(K).right_kernel().basis()
lprint(Kperp)

$ \left[\left(1,\,-\frac{1}{3} \, \sqrt{3},\,0\right), \left(0,\,0,\,1\right)\right] $

In [8]:
Iperp = matrix(I).right_kernel().basis()
lprint(Iperp)

$ \left[\left(1,\,\sqrt{3},\,0\right)\right] $

#### Getting the A,B matrices

In [9]:
A = matrix(K+Kperp).inverse()
lprint(A) # inverse so is change of basis

$ \left(\begin{array}{rrr}
\frac{1}{4} & \frac{3}{4} & 0 \\
\frac{1}{4} \, \sqrt{3} & -\frac{1}{4} \, \sqrt{3} & 0 \\
0 & 0 & 1
\end{array}\right) $

In [10]:
B = matrix(I + Iperp).inverse()
lprint(B)

$ \left(\begin{array}{rrr}
\frac{3}{4} & 0 & \frac{1}{4} \\
-\frac{1}{4} \, \sqrt{3} & 0 & \frac{1}{4} \, \sqrt{3} \\
0 & 1 & 0
\end{array}\right) $

These are the required matrices for the formulation above

#### The new f

Now we can setup a new function $\tilde{f}$ that behaves as we need to apply the implicit function code - in all but the first slot that is

In [11]:
f1A(x,y,z,l) = f1(*list(A*vector([x,y,z])),l)
f2A(x,y,z,l) = f2(*list(A*vector([x,y,z])),l)
f3A(x,y,z,l) = f3(*list(A*vector([x,y,z])),l)
lprint(f1A)
lprint(f2A)
lprint(f2A)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -\frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \cos\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, {\left(x + 3 \, y\right)} \sin\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + e^{z} - 1 $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{4} \, {\left(x + 3 \, y\right)} \cos\left(-\frac{1}{6} \, \pi + {\lambda}\right) + \frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left(-\frac{1}{6} \, \pi + {\lambda}\right) + z $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{4} \, {\left(x + 3 \, y\right)} \cos\left(-\frac{1}{6} \, \pi + {\lambda}\right) + \frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left(-\frac{1}{6} \, \pi + {\lambda}\right) + z $

In [12]:
[ft1,ft2,ft3] = B.inverse()*vector([f1A,f2A,f3A])
lprint(ft1)
lprint(ft2)
lprint(ft3)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -\frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \cos\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, {\left(x + 3 \, y\right)} \sin\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) - \frac{1}{12} \, \sqrt{3} {\left({\left(x + 3 \, y\right)} \cos\left(-\frac{1}{6} \, \pi + {\lambda}\right) + {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left(-\frac{1}{6} \, \pi + {\lambda}\right) + 4 \, z\right)} + e^{z} - 1 $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{64} \, {\left(\sqrt{3} x - \sqrt{3} y\right)}^{3} + \frac{1}{16} \, {\left(x + 3 \, y\right)}^{2} - \frac{1}{6} \, {\left(\pi - 6 \, {\lambda}\right)} z + z^{2} + 4 \, z $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -\frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \cos\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, {\left(x + 3 \, y\right)} \sin\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, \sqrt{3} {\left({\left(x + 3 \, y\right)} \cos\left(-\frac{1}{6} \, \pi + {\lambda}\right) + {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left(-\frac{1}{6} \, \pi + {\lambda}\right) + 4 \, z\right)} + e^{z} - 1 $

In [13]:
Jt = jacobian([ft1,ft2,ft3],(x,y,z))(x=0,y=0,z=0,l=0)
lprint(Jt)

$ \left(\begin{array}{rrr}
0 & \frac{1}{2} \, \sqrt{3} - \frac{1}{2} & -\frac{1}{3} \, \sqrt{3} + 1 \\
0 & 0 & -\frac{1}{6} \, \pi + 4 \\
0 & \frac{1}{2} \, \sqrt{3} + \frac{3}{2} & \sqrt{3} + 1
\end{array}\right) $

In [14]:
Kt = Jt.right_kernel().basis()
lprint(Kt)

$ \left[\left(1,\,0,\,0\right)\right] $

In [15]:
It = Jt.image().basis()
lprint(It)

$ \left[\left(0,\,1,\,0\right), \left(0,\,0,\,1\right)\right] $

Success! - we see that now the kernel and image directions are teased out.

### Running the implicit function code

The implicit function method only works for the case when the $f_{Y}$ partial is invertible, in this case that means we omit $\lambda$ and the kernel direction, which for our $\tilde{f}$ means ingnoring $x$

Similarly we omit $\tilde{f}_{1}$ from consideration from the implicit function theorem, since this traverses the cokernel of the Jacobian

Translating into the language of the implicit function theorem code, somewhat confusingly we set:

$y_{1} = y$

$y_{2} = z$

$x_{1} = \lambda$

$x_{2} = x$


In [16]:
funcs = [ft2, ft3]
position = {'x' : 0, 'y' : 0, 'z' : 0, 'l' : 0} # drawback that have to write this way
var_dict = {'y1' : y, 'y2' : z, 'x1' : l, 'x2' : x}
x_var_keys = ['x1','x2']
x_dim = 2
y_dim = 2

In [17]:
t_dict = TensorDict(funcs, position, var_dict, x_dim, y_dim)

computing Taylor approximaton to 3 order for speedup


In [18]:
out = get_hkx_polynomial(funcs, 3, x_dim, y_dim, var_dict, x_var_keys, t_dict, position)

100%|██████████| 14/14 [00:03<00:00,  4.26it/s]


In [25]:
lprint(out)

$ \left[-\frac{\sqrt{3} {\lambda} x}{\sqrt{3} + 3} - \frac{3 \, x^{3}}{8 \, \pi {\left(\sqrt{3} + 3\right)}} - \frac{3 \, x^{2} {\left(\sqrt{3} + 1\right)}}{4 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{9 \, {\left(\sqrt{3} x^{3} - \frac{16 \, \sqrt{3} {\lambda} x^{2}}{\sqrt{3} + 3} + \frac{16 \, {\lambda} x^{2}}{\pi - 24} - \frac{12 \, x^{3} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{16 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}, \frac{3 \, x^{2}}{8 \, {\left(\pi - 24\right)}} + \frac{9 \, {\left(\sqrt{3} x^{3} - \frac{16 \, \sqrt{3} {\lambda} x^{2}}{\sqrt{3} + 3} + \frac{16 \, {\lambda} x^{2}}{\pi - 24} - \frac{12 \, x^{3} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}}{32 \, {\left(\pi - 24\right)}}\right] $

In [26]:
factorial(t_dict['fXXX'].tensors[0].xy_order.count('X'))

6

In [27]:
lprint(ft2)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{64} \, {\left(\sqrt{3} x - \sqrt{3} y\right)}^{3} + \frac{1}{16} \, {\left(x + 3 \, y\right)}^{2} - \frac{1}{6} \, {\left(\pi - 6 \, {\lambda}\right)} z + z^{2} + 4 \, z $

In [28]:
lprint(t_dict.tfuncs[1])

$ \left(\frac{3}{16 \, \pi}\right) x^{3} + \left(-\frac{3}{4}\right) l^{2} y + \left(\frac{9}{16 \, \pi}\right) x y^{2} + \frac{1}{6} z^{3} + \left(\frac{1}{2} \, \sqrt{3}\right) l x + \frac{1}{2} z^{2} + \left(\frac{1}{2} \, \sqrt{3} + \frac{3}{2}\right) y + \left(\sqrt{3} + 1\right) z $

In [29]:
var_dict

{'y1': y, 'y2': z, 'x1': l, 'x2': x}

This is the parametrisation of the zero set of the 2nd and 3rd compoments of $\tilde{F} = B^{-1} \circ F \circ A$, lets substitute into the first equation, we'll only require the first two terms in its taylor expansion

In [30]:
lprint(taylor(ft1,(x,0), (y,0), (z,0), (l, 0), 4)) # 3rd order seems necessary

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{72} \, \sqrt{3} {\left(\sqrt{3} z^{4} + 2 \, {\lambda}^{3} x\right)} + \frac{1}{6} \, \sqrt{3} {\left(\sqrt{3} z^{2} - {\lambda} x\right)} + \frac{1}{6} \, \sqrt{3} {\left(2 \, z {\left(\sqrt{3} - 1\right)} - y {\left(\sqrt{3} - 3\right)}\right)} + \frac{12 \, \pi {\lambda}^{2} y + 8 \, \pi z^{3} + 9 \, x^{3} + 27 \, x y^{2}}{48 \, \pi} $

In [31]:
#ft1_taylor = ft1.taylor(*[(key,value) for key,value in position.items() if key != 'l'], 2)(l = -pi/6)
ft1_taylor = ft1.taylor(*[(key,value) for key,value in position.items()],4)
# unpacking the position we take the expansion at
lprint(ft1_taylor.polynomial(SR))

$ \left(\frac{1}{36} \, \sqrt{3}\right) l^{3} x + \frac{1}{24} z^{4} + \left(\frac{3}{16 \, \pi}\right) x^{3} + \frac{1}{4} l^{2} y + \left(\frac{9}{16 \, \pi}\right) x y^{2} + \frac{1}{6} z^{3} + \left(-\frac{1}{6} \, \sqrt{3}\right) l x + \frac{1}{2} z^{2} + \left(-\frac{1}{6} \, \sqrt{3} {\left(\sqrt{3} - 3\right)}\right) y + \left(\frac{1}{3} \, \sqrt{3} {\left(\sqrt{3} - 1\right)}\right) z $

In [32]:
ft1_sub = ft1_taylor(y = out[0], z = out[1])
lprint(ft1_sub.polynomial(SR))

$ \left(\frac{2187 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{4}}{128 \, {\left(\pi - 24\right)}^{4}}\right) l^{4} x^{8} + \left(-\frac{2187 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{3}}{512 \, {\left(\pi - 24\right)}^{4}}\right) l^{3} x^{9} + \left(\frac{6561 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{2} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{2}}{16384 \, {\left(\pi - 24\right)}^{4}}\right) l^{2} x^{10} + \left(-\frac{2187 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{3} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{131072 \, {\left(\pi - 24\right)}^{4}}\right) l x^{11} + \left(\frac{2187 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{4}}{8388608 \, {\left(\pi - 24\right)}^{4}}\right) x^{12} + \left(-\frac{729 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{3}}{128 \, {\left(\pi - 24\right)}^{4}}\right) l^{3} x^{8} + \left(\frac{2187 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{2}}{2048 \, {\left(\pi - 24\right)}^{4}}\right) l^{2} x^{9} + \left(-\frac{2187 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{2} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{32768 \, {\left(\pi - 24\right)}^{4}}\right) l x^{10} + \left(\frac{729 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{3}}{524288 \, {\left(\pi - 24\right)}^{4}}\right) x^{11} + \left(\frac{729 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{2}}{1024 \, {\left(\pi - 24\right)}^{4}}\right) l^{2} x^{8} + \left(-\frac{729 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{8192 \, {\left(\pi - 24\right)}^{4}}\right) l x^{9} + \left(\frac{729 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{2}}{262144 \, {\left(\pi - 24\right)}^{4}}\right) x^{10} + \left(-\frac{243 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{3}}{16 \, {\left(\pi - 24\right)}^{3}}\right) l^{3} x^{6} + \left(\frac{729 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{2}}{256 \, {\left(\pi - 24\right)}^{3}}\right) l^{2} x^{7} + \left(-\frac{729 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{2} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{4096 \, {\left(\pi - 24\right)}^{3}} - \frac{81 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{2048 \, {\left(\pi - 24\right)}^{4}}\right) l x^{8} + \left(\frac{243 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{3}}{65536 \, {\left(\pi - 24\right)}^{3}} + \frac{81 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}}{32768 \, {\left(\pi - 24\right)}^{4}}\right) x^{9} + \left(\frac{243 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{2}}{64 \, {\left(\pi - 24\right)}^{3}}\right) l^{2} x^{6} + \left(-\frac{243 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{512 \, {\left(\pi - 24\right)}^{3}}\right) l x^{7} + \left(\frac{243 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{2}}{16384 \, {\left(\pi - 24\right)}^{3}} + \frac{27}{32768 \, {\left(\pi - 24\right)}^{4}}\right) x^{8} + \left(\frac{729 \, {\left(\sqrt{3} + 1\right)}^{2} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{2}}{16 \, \pi {\left(\pi - 24\right)}^{2} {\left(\sqrt{3} + 3\right)}^{2}}\right) l^{2} x^{5} + \left(-\frac{81 \, {\left(\frac{6 \, {\left(\sqrt{3} + 1\right)} {\left(\frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{2}{\pi {\left(\sqrt{3} + 3\right)}}\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{\pi {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{{\left(\pi - 24\right)}^{3}}\right)}}{256 \, \pi}\right) l x^{6} + \left(\frac{81 \, {\left({\left(\frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{2}{\pi {\left(\sqrt{3} + 3\right)}}\right)}^{2} + \frac{\pi {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}}{{\left(\pi - 24\right)}^{3}}\right)}}{4096 \, \pi}\right) x^{7} + \left(\frac{81 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}^{2}}{8 \, {\left(\pi - 24\right)}^{2}} - \frac{81 \, \sqrt{3} {\left(\sqrt{3} + 1\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{8 \, \pi {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}^{2}}\right) l^{2} x^{4} + \left(\frac{27 \, {\left(\frac{\sqrt{3} {\left(\frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{2}{\pi {\left(\sqrt{3} + 3\right)}}\right)}}{\sqrt{3} + 3} - \frac{36 \, {\left(\sqrt{3} + 1\right)}^{2} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{{\left(\pi - 24\right)}^{2} {\left(\sqrt{3} + 3\right)}^{2}}\right)}}{128 \, \pi} - \frac{81 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{64 \, {\left(\pi - 24\right)}^{2}}\right) l x^{5} + \left(\frac{81 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}^{2}}{2048 \, {\left(\pi - 24\right)}^{2}} + \frac{9 \, {\left(\frac{18 \, {\left(\sqrt{3} + 1\right)} {\left(\frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{2}{\pi {\left(\sqrt{3} + 3\right)}}\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{\pi}{{\left(\pi - 24\right)}^{3}}\right)}}{1024 \, \pi}\right) x^{6} + \left(\frac{9 \, {\left(\sqrt{3} + 1\right)} {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{4 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right) l^{3} x^{2} + \left(-\frac{3 \, {\left(\pi {\left(\frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{2}{\pi {\left(\sqrt{3} + 3\right)}}\right)} - \frac{36}{{\left(\sqrt{3} + 3\right)}^{2}}\right)}}{64 \, \pi}\right) l^{2} x^{3} + \left(-\frac{27 \, {\left(\frac{\sqrt{3}}{\sqrt{3} + 3} - \frac{1}{\pi - 24}\right)}}{16 \, {\left(\pi - 24\right)}^{2}} + \frac{27 \, \sqrt{3} {\left(\sqrt{3} + 1\right)}}{32 \, \pi {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}^{2}}\right) l x^{4} + \left(\frac{27 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)}}{256 \, {\left(\pi - 24\right)}^{2}} + \frac{81 \, {\left(\sqrt{3} + 1\right)}^{2}}{256 \, \pi {\left(\pi - 24\right)}^{2} {\left(\sqrt{3} + 3\right)}^{2}}\right) x^{5} + \left(\frac{1}{36} \, \sqrt{3} - \frac{\sqrt{3}}{4 \, {\left(\sqrt{3} + 3\right)}}\right) l^{3} x + \left(-\frac{3 \, {\left(\sqrt{3} + 1\right)}}{16 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right) l^{2} x^{2} + \left(\frac{9}{128 \, {\left(\pi - 24\right)}^{2}}\right) x^{4} + \left(\frac{1}{32} \, \sqrt{3} {\left({\left(\sqrt{3} - 3\right)} {\left(\frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{2}{\pi {\left(\sqrt{3} + 3\right)}}\right)} + \frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} - 1\right)}}{\pi - 24}\right)} + \frac{3}{16 \, \pi}\right) x^{3} + \left(-\frac{1}{6} \, \sqrt{3} + \frac{\sqrt{3} - 3}{2 \, {\left(\sqrt{3} + 3\right)}}\right) l x $

In [34]:
local = sum([a[0]*a[1] for a in ft1_sub.polynomial(SR) if a[1].degree() <= 3])
lprint(local)

$ \left(\frac{1}{32} \, \sqrt{3} {\left({\left(\sqrt{3} - 3\right)} {\left(\frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{2}{\pi {\left(\sqrt{3} + 3\right)}}\right)} + \frac{3 \, {\left(\sqrt{3} - \frac{12 \, {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}\right)} {\left(\sqrt{3} - 1\right)}}{\pi - 24}\right)} + \frac{3}{16 \, \pi}\right) x^{3} + \left(-\frac{1}{6} \, \sqrt{3} + \frac{\sqrt{3} - 3}{2 \, {\left(\sqrt{3} + 3\right)}}\right) l x $

The solution set of this will locally approximate the full solution set, we see that the trivial solution is still preserved for $x,y,z = 0$

We see that $\lambda = 0$ is a potential solution approximation, and will be close to a true bifurcation. Why? regardless of what happens in the higher order terms, the value of $\lambda$ can be adjusted to accomodate, since the $\lambda x$ term dominates close to the bifurcation point

So we see that we can find an approximation of a solution for the set of equations dependant on $\lambda$

Lets try evaluating at $\lambda = 0, x = 0.1$

In [35]:
y_est = out[0](x = 0.1, l = 0)
z_est = out[1](x = 0.1, l = 0)

print(float(ft1(0.1, y_est, z_est, 0)))
print(float(ft2(0.1, y_est, z_est, 0)))
print(float(ft3(0.1, y_est, z_est, 0)))

5.0438346316109796e-05
-7.977956605311235e-06
-1.1749075087450715e-08


A sucess! lets try to incorporate the cubic adjustment

In [36]:
#lprint(local)

In [37]:
better = symbolic_expression(local/x).full_simplify().solve(l)
#lprint(better)

In [39]:
l_est = float(better[0].right()(x = 0.1))
y_est = float(out[0](x = 0.1, l = l_est))
z_est = float(out[1](x = 0.1, l = l_est))

In [40]:
print(float(ft1(0.1, y_est, z_est, l_est)))
print(float(ft2(0.1, y_est, z_est, l_est)))
print(float(ft3(0.1, y_est, z_est, l_est)))

-1.2182126221745829e-08
-6.104203896480586e-06
-1.2456491693456729e-08


Interestingly we see that now only the $\tilde{f}_{1}$ error has significantly reduced, this suggest that for further improvements for the others we need to to a bigger taylor expansion right at the start

## Converting back

Now we need to translate this all back to the original problem

Lets try to paramtrise by $x = t$, using the third order approximation for lambda for most precision available

Take $t = 0.1$

In [41]:
var('t')
l_est = better[0].right()(x = 1/10) # think of x here as t
x_est = t
y_est = out[0](x = t, l = l_est)
z_est = out[1](x = t, l = l_est)
lprint([x_est, y_est, z_est])

$ \left[t, -\frac{3 \, t^{3}}{8 \, \pi {\left(\sqrt{3} + 3\right)}} - \frac{3 \, t^{2} {\left(\sqrt{3} + 1\right)}}{4 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{9 \, {\left(50 \, \sqrt{3} t^{3} - \frac{600 \, t^{3} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{3 \, \sqrt{3} t^{2}}{\pi {\left(\sqrt{3} + 3\right)}} + \frac{3 \, t^{2}}{\pi {\left(\pi - 24\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{800 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{3 \, \sqrt{3} t}{800 \, \pi {\left(\sqrt{3} + 3\right)}}, \frac{3 \, t^{2}}{8 \, {\left(\pi - 24\right)}} + \frac{9 \, {\left(50 \, \sqrt{3} t^{3} - \frac{600 \, t^{3} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{3 \, \sqrt{3} t^{2}}{\pi {\left(\sqrt{3} + 3\right)}} + \frac{3 \, t^{2}}{\pi {\left(\pi - 24\right)}}\right)}}{1600 \, {\left(\pi - 24\right)}}\right] $

This parametrises the solutions of $\tilde{F} = B^{-1} \circ F \circ A$, by invertibility of $B$ we see that these also are the solutions of $F \circ A$, so premultiplication by $A$ will yield the new solution set:

In [42]:
[x_est, y_est, z_est] = list(A*vector([x_est,y_est,z_est]))
lprint([x_est, y_est, z_est])

$ \left[\frac{1}{4} \, t - \frac{9 \, t^{3}}{32 \, \pi {\left(\sqrt{3} + 3\right)}} - \frac{9 \, t^{2} {\left(\sqrt{3} + 1\right)}}{16 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{27 \, {\left(50 \, \sqrt{3} t^{3} - \frac{600 \, t^{3} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{3 \, \sqrt{3} t^{2}}{\pi {\left(\sqrt{3} + 3\right)}} + \frac{3 \, t^{2}}{\pi {\left(\pi - 24\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{3200 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{9 \, \sqrt{3} t}{3200 \, \pi {\left(\sqrt{3} + 3\right)}}, \frac{1}{4} \, \sqrt{3} t + \frac{3}{3200} \, \sqrt{3} {\left(\frac{100 \, t^{3}}{\pi {\left(\sqrt{3} + 3\right)}} + \frac{200 \, t^{2} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{3 \, {\left(50 \, \sqrt{3} t^{3} - \frac{600 \, t^{3} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{3 \, \sqrt{3} t^{2}}{\pi {\left(\sqrt{3} + 3\right)}} + \frac{3 \, t^{2}}{\pi {\left(\pi - 24\right)}}\right)} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} + \frac{\sqrt{3} t}{\pi {\left(\sqrt{3} + 3\right)}}\right)}, \frac{3 \, t^{2}}{8 \, {\left(\pi - 24\right)}} + \frac{9 \, {\left(50 \, \sqrt{3} t^{3} - \frac{600 \, t^{3} {\left(\sqrt{3} + 1\right)}}{{\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}} - \frac{3 \, \sqrt{3} t^{2}}{\pi {\left(\sqrt{3} + 3\right)}} + \frac{3 \, t^{2}}{\pi {\left(\pi - 24\right)}}\right)}}{1600 \, {\left(\pi - 24\right)}}\right] $

In [43]:

x_est1 = x_est(t = 0.1)
y_est1 = y_est(t = 0.1)
z_est1 = z_est(t = 0.1)
l_est1 = better[0].right()(x = 0.1) 
#l_est = 0
print(float(f1(x_est1, y_est1, z_est1, l_est1)))
print(float(f2(x_est1, y_est1, z_est1, l_est1)))
print(float(f3(x_est1, y_est1, z_est1, l_est1)))

-1.2250717600081895e-08
-1.1880373074541124e-10
-6.10420389648064e-06


Perfect - we have translated back, the method documented above allows us to do this to arbitrary precision :)